# 🐣 Document Intelligence with Docling: Unlocking Complex Academic Content

This notebook demonstrates **Document Intelligence** - the advanced capability to understand and process complex documents like research papers, academic materials, and structured content that traditional RAG systems struggle with.

**The Challenge:**
Imagine trying to build an educational AI assistant using only basic text extraction from research papers. You'd lose:
- **📊 Table data** with crucial research findings
- **🧮 Mathematical formulas** and scientific notation  
- **📈 Charts and figures** that provide key insights
- **🏛️ Document structure** like sections, references, and metadata
- **📝 Multi-column layouts** common in academic papers

**The Solution: Docling**
Docling is an advanced document processing toolkit that acts like a brilliant research assistant, understanding the **meaning and structure** of complex academic documents.

**What You'll Build:**
- **🔬 Intelligent Document Processor**: Extract rich content from complex PDFs
- **📚 Enhanced RAG System**: Query tables, formulas, and structured content  
- **🎯 Academic AI Assistant**: Answer questions using complete document understanding
- **⚡ Production Pipeline**: Handle real-world educational materials at scale

**Why This Matters:**
Traditional RAG systems often fail with academic content, missing critical information trapped in tables or losing context from complex layouts. Docling transforms these challenging documents into fully searchable, queryable knowledge.

#### Let's build document intelligence that truly understands academic content! 🚀

## 📦 Install Required Packages

Install the Python packages needed for this lab.

In [ ]:
!pip install -q llama_stack==0.3.0 llama_stack_client==0.3.0 fire==0.7.1 dotenv==0.9.9

## 📚 Import Libraries for Document Intelligence

Import the essential libraries for building our document intelligence RAG system with Docling processing capabilities.

In [ ]:
# Core libraries for document intelligence and RAG
import uuid     
import requests 
import base64   
import json 
import os 
import sys  
from io import BytesIO, StringIO
sys.path.append('..') 

# LlamaStack client and RAG-specific classes
from llama_stack_client import LlamaStackClient  
from llama_stack_client import RAGDocument
from llama_stack.apis.common.content_types import TextDelta, ToolCallDelta

# Display and utility imports
from src.utils import step_printer 
from termcolor import cprint

## 🔗 Connect to LlamaStack

In [ ]:
# The base URL points to your Llama Stack server deployment
base_url = "http://llama-stack-service:8321"

# Create the Llama Stack client
client = LlamaStackClient(
    base_url=base_url,
    provider_data=None
)

print(f"Connected to Llama Stack server")

# Configs for model and sampling
model_id = "llama32"
temperature = 0.0
max_tokens = 512
stream = False

print(f"Model: {model_id}")
print(f"Stream: {stream}")

## 🔬 Docling Processing Function Implementation

This function connects to the Docling service and processes documents through the intelligent pipeline we just described.  
Let's test Docling's document intelligence on a complex academic paper. We'll use a real research paper that contains tables, mathematical formulas, and figures 📈

> Note: Because Docling is using more advanced document extraction it usually takes a little bit to extract the information.  
This particular pdf we use should take 1-2 minutes.

In [ ]:
def docling_processing(url):

    # Connect to the deployed Docling service in the cluster
    api_address = "http://docling-v0-7-0-predictor.ai501.svc.cluster.local:5001"
    headers = {"Content-Type": "application/json"}

    print(f"🔗 Connecting to Docling service at {api_address}")
    
    # Docling settings for processing
    payload = {
        "http_sources": [{"url": url}],
        "options": {
            "to_formats": ["md"],
            "image_export_mode": "placeholder"
        },
    }
    
    try:
        # Send document to Docling for intelligent analysis
        response = requests.post(
            f"{api_address}/v1alpha/convert/source",
            json=payload,
            headers=headers,
            timeout=180
        )
        
        response.raise_for_status()
        
        result_data = response.json()
        md_content = result_data["document"]["md_content"]
        
        return md_content
        
    except requests.exceptions.Timeout:
        print(f"⏰ Processing timeout - complex documents may need more time")
        raise
    except requests.exceptions.RequestException as e:
        print(f"❌ Docling processing failed: {e}")
        raise
    except KeyError as e:
        print(f"❌ Unexpected response format: {e}")
        raise

In [ ]:
# We choose a research paper called "First Mapping the Canopy Height of Primeval Forests in the Tallest Tree Area of Asia", fitting for our Canaopy application.
url = "https://arxiv.org/pdf/2404.14661"

md_content = docling_processing(url)

print(f"\n🎉 Document intelligence processing complete!")
print(f"📊 Content preview (first 500 characters):")
print(f"{'='*60}")
print(md_content[:500] + "..." if len(md_content) > 500 else md_content)
print(f"{'='*60}")
print(f"📈 Total processed content: {len(md_content)} characters")
print(f"📝 Docling has extracted and structured the complete document content!")

## 📃 Using the processed document for RAG

We can now add this document to our Milvus database (the in-line one, remember that we are in the experiment namespace and connect to the experiment Llama Stack for now).  
Since we have already done this once you will probably recognize some parts of the code.

In [ ]:
# Connect to the vector database
vs = client.vector_stores.create(
    name="test_docling_store",
    extra_body={
        "embedding_model": "all-MiniLM-L6-v2",
        "embedding_dimension": 384,
        "provider_id": "milvus",
        "vector_db_id": "test-docling"
    }
)
vector_db_id = vs.id
print(f"📊 Created vector database with ID: {vector_db_id}")

In [ ]:
# Upload the Docling-processed markdown content as a file and add to vector store
try:
    print("📤 Uploading processed markdown content as file...")
    
    # Create a file-like object from the markdown content
    file_content = BytesIO(md_content.encode('utf-8'))
    file_content.name = "docling-processed-research-paper.md"
    file_content.source_url =  url,
    file_content.processing_method = "docling",
    file_content.document_type = "academic_paper",
    file_content.has_tables = True,
    file_content.has_formulas = True,
    file_content.has_figures = True,
    
    # Upload file using the files API
    uploaded_file = client.files.create(
        file=file_content,
        purpose="assistants"  # Required purpose parameter
    )
    
    print(f"✅ Uploaded processed content as file with ID: {uploaded_file.id}")
    
    # Add the file to the vector store with chunking configuration
    client.vector_stores.files.create(
        vector_store_id=vector_db_id,
        file_id=uploaded_file.id,
        chunking_strategy={
            "type": "static",
            "static": {
                "max_chunk_size_tokens": 512,
                "chunk_overlap_tokens": 50
            }
        }
    )
    
    print(f"✅ Added processed file to vector store with chunking")
    print(f"\n✅ Document ingestion complete!")
    print(f"🎯 Docling-processed content is now searchable via semantic similarity!")
    
except Exception as e:
    print(f"\n❌ Document ingestion failed: {e}")
    print("💡 This might be due to file processing issues. Check the markdown content or try again.")

In [ ]:
# Ingest the document as processed by Docling
documents = [
    RAGDocument(
        document_id=f"docling-processed-doc",
        content=md_content,
        metadata={
            "source_url": url,
            "processing_method": "docling",
            "document_type": "academic_paper",
            "has_tables": True,
            "has_formulas": True,
            "has_figures": True,
        },
    )
]

try:
    client.tool_runtime.rag_tool.insert(
        documents=documents,
        vector_db_id=vector_db_id,
        chunk_size_in_tokens=512,
    )
    print(f"\n✅ Document ingestion complete!")
    
except Exception as e:
    print(f"\n❌ Document ingestion failed: {e}")

Now that we have ingested the document (added it into our Vector Database) we can query for it just like we did before!  
Feel free to play around with different queries to see what it answers.

In [ ]:
# Test queries for the processed document
queries = [
    "What is the PRFXception?",
    "The accuracy values of overall model prediction and residual cross-validation for five regions in southeast Tibet and four regions in northwest Yunnan"
]

for prompt in queries:
    cprint(f"\nUser> {prompt}", "blue")
    
    # RAG retrieval call - find relevant chunks from the vector database
    search_results = client.vector_stores.search(
        vector_store_id=vs.id,
        query="What is the PRFXception?",
        max_num_results=5,
        search_mode="vector"  # Use vector similarity search
    )

    cprint(search_results)

    exclude = ["processing_method", "document_id", "token_count", "metadata_token_count"]
    retrieved_chunks = []
    cprint(f"\n--- RAG Metadata ---", "yellow")
    for i, result in enumerate(search_results.data): 
        metadata = {k: v for k, v in search_results.data[0].attributes.items() if k not in exclude}
        cprint(metadata, "cyan")
        retrieved_chunks.append(result.content[0].text)


    # Create messages for the LLM with system prompt
    messages = [{"role": "system", "content": "You are a helpful assistant."}]

    # Combine the user query with retrieved context from RAG
    prompt_context = "\n\n".join(retrieved_chunks)
    extended_prompt = f"Please answer the given query using the context below.\n\nCONTEXT:\n{prompt_context}\n\nQUERY:\n{prompt}"
    messages.append({"role": "user", "content": extended_prompt})

    # Get response from the LLM using the enhanced prompt
    response = client.chat.completions.create(
        model=model_id,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature,
        stream=True,
    )

    # Print the streaming response
    cprint("inference> ", color="magenta", end='')
    
    # Handle streaming response - tokens arrive one by one
    for chunk in response:
        if hasattr(chunk, 'choices') and chunk.choices:
            delta = chunk.choices[0].delta
            if hasattr(delta, 'content') and delta.content:
                cprint(delta.content, color="magenta", end='')
        elif hasattr(chunk, 'content'):
            cprint(chunk.content, color="magenta", end='')

## 🎉 You have used Docling to enhance your document processing!

Your document intelligence system can now understand and query the most complex academic content - transforming how educational institutions handle knowledge discovery and research! 🚀  
Go back to the instructions to see how we can automate our document ingestion.